# Exploration phase — initial testing
Here we try out different models on our data. The comments below explain exactly what is happening.

In [ ]:
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
from pathlib import Path
import pandas as pd
from sklearn.base import clone
import time

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [ ]:
project_path = Path.cwd().parent
bootstrap_project(project_path)
print(f"Ścieżka projektu: {project_path}")

In [ ]:
with KedroSession.create(project_path=project_path) as session:
    context = session.load_context()
    dfs = context.catalog.load("circuit_lap_data")

print(f"Załadowano dane dla {len(dfs)} obwodów:")
for circuit, df in dfs.items():
    print(f"  {circuit}: {df.shape[0]} okrążeń, {df.shape[1]} cech")

In [ ]:
# Prepare regressor configurations for testing

# We test many algorithms with parameter tuning using GridSearchCV.
# The GridSearchCVs here will be used as templates. For each circuit,
# every of the GridSearchCVs below will be cloned and fitted to their data.

# GridSearchCV configurations
model_searches = {
    # Linear regression
    "LinearRegression": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), LinearRegression()),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "RidgeCV": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), RidgeCV(alphas=(0.1, 1.0, 10.0))),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "LassoCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            LassoCV(max_iter=100_000, alphas=[0.001, 0.01, 0.1, 1.0]),
        ),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "ElasticNetCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            ElasticNetCV(max_iter=100_000, l1_ratio=[0.2, 0.5, 0.8]),
        ),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    # Polynomial regression
    "PolynomialLinearRegression": GridSearchCV(
        make_pipeline(
            StandardScaler(), PCA(), PolynomialFeatures(), LinearRegression()
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialRidgeCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            RidgeCV(alphas=(0.1, 1.0, 10.0)),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialLassoCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            LassoCV(max_iter=100_000, alphas=[0.001, 0.01, 0.1]),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialElasticNetCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            ElasticNetCV(max_iter=100_000, l1_ratio=[0.2, 0.5, 0.8]),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    # Bagging models
    "RandomForestRegressor": GridSearchCV(
        RandomForestRegressor(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [5, 10, 20, None],
            "min_samples_split": [2, 5, 10],
        },
    ),
    "ExtraTreesRegressor": GridSearchCV(
        ExtraTreesRegressor(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [5, 10, 20, None],
            "min_samples_split": [2, 5, 10],
        },
    ),
    # Boosting models
    "AdaBoostRegressor": GridSearchCV(
        AdaBoostRegressor(random_state=42),
        {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 0.5, 1.0]},
    ),
    "GradientBoostingRegressor": GridSearchCV(
        GradientBoostingRegressor(random_state=42),
        {
            "n_estimators": [100, 200],
            "learning_rate": [0.01, 0.05, 0.1],
            "max_depth": [3, 5],
            "subsample": [0.8, 1.0],
        },
    ),
    "XGBRegressor": GridSearchCV(
        XGBRegressor(
            random_state=42, n_jobs=-1, objective="reg:squarederror", verbosity=0
        ),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [3, 6, 10],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
        },
    ),
    # Support vector models
    "SVR_linear": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), SVR(kernel="linear")),
        {"svr__C": [0.1, 1, 10, 100], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "SVR_rbf": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), SVR(kernel="rbf")),
        {
            "svr__C": [0.1, 1, 10],
            "svr__gamma": ["scale", 0.01, 0.1, 1.0],
            "pca__n_components": [0.98, 0.95, 0.9],
        },
    ),
    # MLP
    "MLPRegressor": GridSearchCV(
        make_pipeline(
            StandardScaler(), PCA(), MLPRegressor(max_iter=100_000, random_state=42)
        ),
        {
            "mlpregressor__hidden_layer_sizes": [
                (16,),
                (24,),
                (24, 12),
                (16, 16),
                (16, 8),
            ],
            "mlpregressor__activation": ["relu", "tanh"],
            "mlpregressor__alpha": [0.0001, 0.001, 0.01],
            "mlpregressor__learning_rate_init": [0.001, 0.01],
            "pca__n_components": [0.98, 0.95, 0.9],
        },
    ),
}

In [ ]:
# Fit every single circuit/GridSearch configuration
models_and_circuits = {}

for name in model_searches.keys():
    models_and_circuits[name] = {}

for circuit, data in dfs.items():
    print(f"Fitting models for {circuit}")
    circuit_start = time.time()

    X, y = data.drop(["LapTimeZScore"], axis="columns"), data["LapTimeZScore"]
    for name, model_search in model_searches.items():
        print(f"Fitting {name};".ljust(50), end="")
        model_start = time.time()

        model_search_copy = clone(model_search)
        model_search_copy.fit(X, y)
        models_and_circuits[name][circuit] = model_search_copy

        print(f"took {round(time.time() - model_start, 2)} seconds")

    print(
        f'Took a total of {round(time.time() - circuit_start, 2)} seconds to fit all models for circuit "{circuit}"'
    )
    print()

In [ ]:
# Save models for later use
context.catalog.save("initial_models", models_and_circuits)
print("Zapisano modele")

In [ ]:
# Show scores for each GridSearch and circuit
all_scores = {}
for key in models_and_circuits.keys():
    scores = {}
    for circuit, model in models_and_circuits[key].items():
        scores[circuit] = model.best_score_
    all_scores[key] = scores

all_scores = pd.DataFrame(all_scores)

all_scores

In [ ]:
# Show score statistics for each model
# MinScore is very important. A good model should perform reasonably well for all tracks.
model_scores_df = pd.DataFrame(
    {
        "MeanScore": all_scores.mean(axis="index"),
        "MedianScore": all_scores.median(axis="index"),
        "ScoreVariance": all_scores.var(axis="index"),
        "MinScore": all_scores.min(axis="index"),
    }
)

model_scores_df.sort_values(by=["MeanScore"], ascending=False)

## Result interpretation
### The top-2
**XGBRegressor is a clear winner**. The lowest score it got is over 0.64, mean and median scores are highest of all models, while score variance is low. It is clear that this algorithm reliably provides good results.

**GradientBoostingRegressor** is a close runner up, with similar characteristics, albeit somewhat less accurate and less consistent. This does not come as a surprise, since it uses a similar but less advanced algorithm to XGBoost. 

### Remaining results
The rest of the models have serious flaws. For example, **RandomForestRegressor**, despite having decent overall scores, has a higher score variance and got a score below 0.2 for one of the tracks. **ExtraTreesRegressor** is better in that regard, but still inferior to out top-2 models. 

The rest of the regressors perform significantly worse than the others, with versions of polynomial and linear regression having particularly low performance. There are some outlying values, even negative ones, in these models. Considering that XGBoost is a clear winner, I do not deem it necessary to look into this further at this point.

### To sum up
It appears that *boosting models*, particularly XGBRegressor and GradientBoostingRegressor, are the best. These are the models that will be optimized and tested further.

<br><br><br>


In [ ]:
# Show how the best models perform on every circuit
relevant_scores = all_scores.loc[:, ["XGBRegressor", "GradientBoostingRegressor"]]
track_scores_df = pd.DataFrame(
    {
        "MeanScore": relevant_scores.mean(axis="columns"),
        "XGBRegressorScore": relevant_scores["XGBRegressor"],
        "GradientBoostingRegressorScore": relevant_scores["GradientBoostingRegressor"],
        "DataPointCount": [df.shape[0] for df in dfs.values()],
    }
)
track_scores_df.sort_values(by=["MeanScore"])

## About scores by circuit
Scores clearly vary a lot depending on the circuit. It is important to note that both the characteristics of the circuit itself, as well as how much data we have on each circuit has a big effect. For some circuits we only have data from one session, which is an obvious limitation and could affect score in different ways. The score tends to be higher for circuits with less than 2000 data points, possibly because the data points only come from one or two sessions in those cases. This makes it very likely for weather to be roughly constant throughout the data relevant to them, skewing CV results in favour of the model.

### Point in favour of the results
Even in the worst case, XGBoost had a mean score of over 0.64, which means it accounted for 64% of target attribute variance. <br> 
The target attribute in this case the driver's lap time z-score within each session. Z-score in this case basically denotes how good the lap was compared to the other laps the same driver completed in the same session. This means that for the most unpredictable circuit, our model accounted for 64% of how pit stops and weather affect driver performance. For over 70% of the circuits, the model accounted for over 80% of those differences.

Therefore, it is clear to me that boosting models can produce decent-to-excellent results in general, even if some scores are exaggerated due to insufficient data size.